In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
raw=pd.read_csv('clean_data.csv', index_col= 0)

In [20]:
target=raw.label

In [3]:
raw.head()

,text,label
0,need add water drug vaccine ready administered...,0
1,hydroxychloroquine shown effective rate treati...,0
2,fact hydroxychloroquine shown effective rate t...,0
3,man made created wuhan laboratory ask billgate...,0
4,billgates finance research wuhan lab created g...,0


In [4]:
text_list=raw['text'].to_list()

In [5]:
text_list

['need add water drug vaccine ready administered two part kit one hold pellet containing chemical machinery synthesis end product hold pellet containing instruction telll drug compound create mix two part together chosen combination add water treatment ready',
 'hydroxychloroquine shown effective rate treating yet democrat gretchen whitmer threatening doctor prescribe trump something democrat okay dying mean opposing trump',
 'fact hydroxychloroquine shown effective rate treating yet democrat gretchen whitmer threatening doctor prescribe trump something democrat okay dying mean opposing trump sick',
 'man made created wuhan laboratory ask billgates financed',
 'billgates finance research wuhan lab created georgesoros good friend gate',
 'urgent health bulletin public ministry health emergency notification public outbreak time serious fatal cure infected spreading china various country prevention method keep throat moist let throat dry thus hold thirst membrane throat dried invade body 

In [24]:
training_sentences = text_list[0:865]
testing_sentences = text_list[865:]
training_labels = target[0:865]
testing_labels = target[865:]

In [31]:
tokenizer = Tokenizer(num_words=2000, oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)
#make sure the neural network only see the train data
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, padding='post', maxlen= 1600, truncating= 'post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, padding='post',maxlen= 1600, truncating= 'post')

In [32]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [40]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(2000, 16, input_length=1600),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1600, 16)          32000     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               2176      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)               

In [44]:
num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/50
28/28 - 0s - loss: 9.1365e-04 - accuracy: 1.0000 - val_loss: 0.4057 - val_accuracy: 0.9196
Epoch 2/50
28/28 - 0s - loss: 9.1427e-04 - accuracy: 1.0000 - val_loss: 0.4153 - val_accuracy: 0.9161
Epoch 3/50
28/28 - 0s - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.5712 - val_accuracy: 0.8951
Epoch 4/50
28/28 - 0s - loss: 0.0176 - accuracy: 0.9954 - val_loss: 0.4474 - val_accuracy: 0.9126
Epoch 5/50
28/28 - 0s - loss: 0.0125 - accuracy: 0.9977 - val_loss: 0.8579 - val_accuracy: 0.8217
Epoch 6/50
28/28 - 0s - loss: 0.0136 - accuracy: 0.9977 - val_loss: 0.5168 - val_accuracy: 0.8951
Epoch 7/50
28/28 - 0s - loss: 0.0035 - accuracy: 0.9988 - val_loss: 0.4058 - val_accuracy: 0.9161
Epoch 8/50
28/28 - 0s - loss: 7.5063e-04 - accuracy: 1.0000 - val_loss: 0.3929 - val_accuracy: 0.9301
Epoch 9/50
28/28 - 0s - loss: 8.7620e-04 - accuracy: 1.0000 - val_loss: 0.4038 - val_accuracy: 0.9266
Epoch 10/50
28/28 - 0s - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.4042 - val_accuracy: 0.9301
Epo